In [80]:
import requests
from bs4 import BeautifulSoup as Soup
import brotli
import pandas as pd
import json
import string

In [2]:
url = "http://fr.indeed.com/emplois?q=data+engineer&l=Paris&radius=10"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Connection": "keep-alive",
    "Accept-Language": "en-US,en;q=0.9,lt;q=0.8,et;q=0.7,de;q=0.6",
}

In [68]:
df = pd.DataFrame({}, columns=["title", "company", "city", "min_salary", "max_salary", "frequency"])

resp = requests.get(url, headers=headers)
# Détecter l'encodage spécifié dans les headers
content_type = resp.headers.get('Content-Type', '')
print(content_type)
encoding = 'utf-8'  # Encodage par défaut
if 'charset=' in content_type:
    encoding = content_type.split('charset=')[-1]
# Décompresser le contenu si encodé en Brotli
if resp.headers.get('Content-Encoding') == 'br':
    try:
        decompressed_content = brotli.decompress(resp.content)
        text = decompressed_content.decode(encoding)
    except Exception as e:
        print("Error decompressing content:", e)
        text = resp.text
else:
    text = resp.text  # Utiliser la méthode text qui gère automatiquement l'encodage
# Parser le contenu HTML avec BeautifulSoup
soup = Soup(text, "html.parser")

contents = soup.find_all("li", {"class": "css-5lfssm eu4oa1w0"})

for element in contents[:-5]:
    element_dict={}
    head = element.find("h2", {"class": "jobTitle css-198pbd eu4oa1w0"})
    location = element.find("div", {"class": "company_location css-17fky0v e37uo190"})
    if head is not None:
        title = head.find("span")["title"]
        element_dict["title"] = title
    else:
        continue
    if location is not None:
        company_name = location.find("span", {"data-testid":"company-name" , "class": "css-63koeb eu4oa1w0"})
        if company_name is not None:
            element_dict["company"] = company_name.text
        city = location.find("div", {"data-testid":"text-location" , "class": "css-1p0sjhy eu4oa1w0"})
        if city is not None:
            element_dict["city"] = city.text
    metadata = element.find("div", {"class":"heading6 tapItem-gutter metadataContainer css-z5ecg7 eu4oa1w0"})
    if metadata is not None:
        salary_block = metadata.find("div", {"class":"metadata salary-snippet-container css-5zy3wz eu4oa1w0"})
        if salary_block is not None:
            salary = salary_block.find("div", {"data-testid":"attribute_snippet_testid", "class":"css-1cvvo1b eu4oa1w0"})
            boundaries = salary.text.split("€")[:-1]
            frequency = salary.text.split(" ")[-1]
            frequency_map = {"mois":"mensuel", "an":"annuel"}
            element_dict["frequency"] = frequency_map.get(frequency, "autre")
            for boundary in boundaries:
                if "de" in boundary.lower():
                    element_dict["min_salary"] = "".join(filter(str.isdigit, boundary))
                elif "à" in boundary.lower():
                    element_dict["max_salary"] = "".join(filter(str.isdigit, boundary))

    grade_span = element.find("span", {"data-testid":"holistic-rating"})
    if grade_span is not None:
        grade = grade_span["aria-label"].split(" ")[0]
    
    url_job = "http://fr.indeed.com" + element.find("a", {"class":"jcs-JobTitle css-jspxzf eu4oa1w0"})["href"]
    if any(element_dict.values()):
        df = pd.concat([df, pd.DataFrame(element_dict, index=[0])], ignore_index=True)

text/html;charset=UTF-8
Error decompressing content: BrotliDecompress failed


In [6]:
df.to_csv("data.csv", index=False, sep=";", encoding="utf-8")

In [91]:
print(url_job)
response = requests.get(url_job, headers=headers)
response.raise_for_status()
html_content = response.content.decode("utf-8")
soup = Soup(html_content, "html.parser")
script = [s for s in soup.find_all("script") if "datePublished" in str(s)][0]

kw = json.loads(open("resources/kw_list.json").read())

description = soup.find("div", {"id": "jobDescriptionText","class":"jobsearch-JobComponent-description css-16y4thd eu4oa1w0"})
description_text = description.text.lower()
description_text = description_text.translate(str.maketrans('', '', string.punctuation))

if description is not None:
    words = description_text.split(" ")
    for key in kw:
        for word in kw[key]:
            if word in words:
                print(f"{word} found in description of job {title}")
    

http://fr.indeed.com/rc/clk?jk=f9fcb0cc4bf45e68&bb=yBpBdasUGwFfA30zGFo6JpQRaxGbB_m-ESA-YwOTKwibgqY9Nn72hNbKTiejqjIir4-lDbhcgoNUw6kNovr2AI88QfU1BCR4ublrsErzB_0llyaZXd2R2uzwxM7w81lA&xkcb=SoBd67M3A2T_FFyFiZ0BbzkdCdPP&fccid=f34c5c7a544de9ba&cmp=Cyber-University&ti=Ing%C3%A9nieur+Cloud&vjs=3
python found in description of job Cloud Data Engineer - POEI
azure found in description of job Cloud Data Engineer - POEI
aws found in description of job Cloud Data Engineer - POEI
gcp found in description of job Cloud Data Engineer - POEI
scala found in description of job Cloud Data Engineer - POEI
informatique found in description of job Cloud Data Engineer - POEI
java found in description of job Cloud Data Engineer - POEI
